# Extracting weather data using web API - Context Data Preprocessing

First, this scrip contains the code that has been written to scrap  weather data from the web API : http://api.wunderground.com. 
 
The tidy data will be handled throught a data preprocessing step so that it would be ready to run with our training model.

The location and the period pre-selected for this example are  Bordeaux and from 01/11/2017 to 02/11/2017 but these parameters can be tuned as you need.

The code has been scheduled so that it is executed every 2 minutes since the  maximum number of queries is limited by the data owner.

The result will be exported and saved under a csv file that can be directly put into our training model as one of inputs.


In [1]:
import numpy as np
import pandas as pd
import time
import datetime 
from pandas.io.json import json_normalize
import json, requests
from urllib.request import urlopen
import csv
import smtplib 

Put dates of the period for which you want to get data under this format: YYYY, MM, DD

In [2]:
start_year=int(input())

2017


In [3]:
start_month=int(input())

11


In [4]:
start_date=int(input())

1


In [5]:
end_year=int(input())

2018


In [6]:
end_month=int(input())

3


In [7]:
end_date=int(input())

20


In [8]:
start_period = datetime.date(start_year,start_month,start_date)
end_period   = datetime.date(end_year,end_month,end_date)
dates_string = [ start_period + datetime.timedelta(n) for n in range(int ((end_period - start_period).days))]

In [9]:
dates_string=[dates_string[i].strftime('%Y%m%d') for i in range(len(dates_string))]

Keep only the 2 first days of the period to make sense in this example

In [10]:
ds=dates_string[:3]

In [11]:
ds

['20171101', '20171102', '20171103']

In [12]:
# Create functions 

def replacesunday(row):
    if row == 8:
        return 1
    else:
        return row

def sendemail(from_addr, to_addr_list, cc_addr_list,
              subject, message,
              login, password,
              smtpserver='smtp.gmail.com:xxx'):
    header  = 'From: %s\n' % from_addr
    header += 'To: %s\n' % ','.join(to_addr_list)
    header += 'Cc: %s\n' % ','.join(cc_addr_list)
    header += 'Subject: %s\n\n' % subject
    message = header + message
 
    server = smtplib.SMTP(smtpserver)
    server.starttls()
    server.login(login,password)
    problems = server.sendmail(from_addr, to_addr_list, message)
    server.quit()
    return problems
# sources: http://rosettacode.org/wiki/Send_email#Python


a_day_off=['2016-03-28','2016-05-01','2016-05-05','2016-05-08','2016-05-16','2016-07-14','2016-08-15','2016-11-01',
          '2016-11-11','2017-04-17','2017-05-01','2017-05-08','2017-05-25','2017-06-05','2017-07-14',
           '2017-08-15','2017-11-01','2017-11-11','2018-01-01','2018-04-02','2018-05-01','2018-05-08','2018-05-10',
          '20018-05-21','2018-07-14','2018-08-15','2018-11-01','2018-11-11','2018-12-25']
before_a_day_off=['2016-03-27','2016-04-30','2016-05-04','2016-05-07','2016-05-15','2016-07-13','2016-08-14','2016-10-31',
          '2016-11-10','2017-04-16','2017-04-30','2017-05-07','2017-05-24','2017-06-04','2017-07-13',
           '2017-08-14','2017-10-31','2017-11-10','2017-12-31','2018-04-01','2018-04-30','2018-05-07','2018-05-09',
          '20018-05-20','2018-07-13','2018-08-14','2018-11-01','2018-11-10','2018-12-24']

def schedule(row):
    
    global val
    
    if row.strftime('%Y-%m-%d') in (a_day_off):
        val='a_day_off'
    elif row.strftime('%Y-%m-%d') in (before_a_day_off):
        val='before_a_day_off'
    elif datetime.date(2016,2,13)<row.date()<datetime.date(2016,2,29):
        val='holidays'
    elif datetime.date(2016,4,9)<row.date()<datetime.date(2016,4,25):
        val='holidays'
    elif datetime.date(2016,5,4)<row.date()<datetime.date(2016,5,9):
        val='holidays'
    elif datetime.date(2016,7,5)<row.date()<datetime.date(2016,9,1):
        val='holidays'
    elif datetime.date(2016,10,19)<row.date()<datetime.date(2016,11,3):
        val='holidays'
    elif datetime.date(2016,12,17)<row.date()<datetime.date(2017,1,3):
        val='holidays'
    elif datetime.date(2017,2,18)<row.date()<datetime.date(2017,3,6):
        val='holidays'
    elif datetime.date(2017,4,15)<row.date()<datetime.date(2017,5,2):
        val='holidays'
    elif datetime.date(2017,5,24)<row.date()<datetime.date(2017,5,29):
        val='holidays'
    elif datetime.date(2017,7,8)<row.date()<datetime.date(2017,9,4):
        val='holidays'
    elif datetime.date(2017,10,21)<row.date()<datetime.date(2017,11,6):
        val='holidays'
    elif datetime.date(2017,12,23)<row.date()<datetime.date(2018,1,8):
        val='holidays'
    elif datetime.date(2018,2,10)<row.date()< datetime.date(2018,2,26):
        val='holidays'
    elif datetime.date(2018,4,7)<row.date()<datetime.date(2018,4,23):
        val= 'holidays'
    elif datetime.date(2018,7,7)<row.date()<datetime.date(2018,9,3):
        val='holidays'
    elif datetime.date(2018,10,20)< row.date()< datetime.date(2018,11,5):
        val= 'holidays'
    elif datetime.date(2018,12,22)<row.date()<datetime.date(2019,1,7):
        val='holidays'
    else:
        val='no_event'
    return val
        

In [13]:
#In order to stay in touch with the execution of this code , we are going to send an email informing the start and another email for the end of the program. = 'atophan06@gmail.com',
sendemail(from_addr    = 'youremail@gmail.com', 
          to_addr_list = ['youremail@xxx.fr'],
          cc_addr_list = ['youremail@xxx.fr'], 
          subject      = ' Web scraping with python code', 
          message      = 'Howdy from a python function: " Web scraping to get weather data has just started "', 
          login        = 'youremail@gmail.com', 
          password     = 'yourpassword')
starttime=time.time()
while True:
    with open(r'dates.csv', 'r') as f:
        reader = csv.reader(f)
        dat = list(reader)
        row_num = len(dat)
       
    with open(r'dates.csv', 'a',newline='') as g, open(r'output.csv', 'a',newline='') as h :
        writer = csv.writer(g) 
        #writerh = csv.writer(h)
        if row_num <= len(ds):
            writer.writerow([ds[row_num-1]])
            url = ("http://api.wunderground.com/api/PUTYOURKEYHERE/history_/q/FR/Bordeaux.json" %ds[row_num-1] )
            data=list()
            dat1 = requests.get(url).json()  
            data.append(json_normalize(dat1['history']['observations']))
            data=pd.concat([data[i]for i in range(len(data))],axis=0, ignore_index=True)
            data=data[['date.hour', 'date.min','date.mday', 'date.mon','date.pretty', 'date.year','tempm','hum','wdire','conds']]
            data= data.copy()
            data['date.pretty']=pd.to_datetime(data['date.pretty'])
            data['DoW']=[data['date.pretty'][i].weekday()+2 for i in range(len(data['date.pretty']))]
            data['DoW']=data['DoW'].apply(replacesunday)
            data['WoY']=[data['date.pretty'][i].strftime("%U") for i in range(len(data['date.pretty']))]
            data['QoY']=[data['date.pretty'][i].to_period('Q') for i in range(len(data['date.pretty']))]
            data['HMS']=[data['date.pretty'][i].strftime("%H:%M:%S") for i in range(len(data['date.pretty']))]
            data['Date']=[data['date.pretty'][i].strftime("%Y-%m-%d") for i in range(len(data['date.pretty']))]
            data=data.rename(columns={'date.hour':'H','date.mday':'DoM','date.mon':'MoY','date.pretty':'Time','tempm':'TemperatureC',
                  'hum':'Humidity','wdire':'Wind_Direction','conds':'Conditions'})
            data=data[['Time','TemperatureC','Humidity','Wind_Direction','Conditions','Date','DoW','DoM','WoY','MoY','QoY','H','HMS']]
            data['Date']=pd.to_datetime(data['Date'])
            data['schedule']=data['Date'].apply(schedule)
            if row_num==1:
                data.to_csv(h, header=True, index=False)
            else:
                data.to_csv(h, header=False, index= False)
            time.sleep(120)
            continue
        else:
            timedelta=(time.time()-starttime)/60
            msg='Howdy from a python function: " The weather data is now ready. The duration of code execution is %d  minutes. "' %timedelta
            sendemail(from_addr    = 'youremail@gmail.com', 
                      to_addr_list = ['youremail@xxx.fr'],
                      cc_addr_list = ['youremail@xxx.fr'], 
                      subject      = ' Web scraping with python code', 
                      message      = msg 
                      login        = 'youremail@gmail.com', 
                      password     = 'yourpassword')
            print( 'End of Web scraping action')
            break
    f.close() 
    g.close()
    h.close()
    


End of Web scraping action


In [14]:
df= pd.read_csv('output.csv')

In [15]:
df.head(2)

,Time,TemperatureC,Humidity,Wind_Direction,Conditions,Date,DoW,DoM,WoY,MoY,QoY,H,HMS,schedule
0,2017-11-01 00:00:00,7.0,73,East,Clear,2017-11-01,4,1,44,11,2017Q4,0,00:00:00,a_day_off
1,2017-11-01 00:00:00,7.0,76,East,Clear,2017-11-01,4,1,44,11,2017Q4,0,00:00:00,a_day_off


In [16]:
df.tail(2)

,Time,TemperatureC,Humidity,Wind_Direction,Conditions,Date,DoW,DoM,WoY,MoY,QoY,H,HMS,schedule
198,2017-11-03 23:00:00,11.0,94,North,Patches of Fog,2017-11-03,6,3,44,11,2017Q4,23,23:00:00,holidays
199,2017-11-03 23:30:00,10.0,94,Variable,Patches of Fog,2017-11-03,6,3,44,11,2017Q4,23,23:30:00,holidays


In [17]:
df.shape

(200, 14)